In [199]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRFRegressor
from sklearn.metrics import mean_squared_error

In [232]:
df_train = pd.read_csv('./data/train.csv')
df_test = pd.read_csv('./data/test.csv')

In [233]:
df_train.head()

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0


In [239]:
df_train.shape[0]

230130

In [241]:
df_train.iloc[:df_train.shape[0]]['id'].values

array([     0,     65,     64, ..., 230063, 230060, 230129], dtype=int64)

In [234]:
df_total = pd.concat([df_train, df_test], axis=0)
df_total

,id,date,country,store,product,num_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,NaN
1,1,2010-01-01,Canada,Discount Stickers,Kaggle,973.0
2,2,2010-01-01,Canada,Discount Stickers,Kaggle Tiers,906.0
3,3,2010-01-01,Canada,Discount Stickers,Kerneler,423.0
4,4,2010-01-01,Canada,Discount Stickers,Kerneler Dark Mode,491.0
...,...,...,...,...,...,...
98545,328675,2019-12-31,Singapore,Premium Sticker Mart,Holographic Goose,NaN
98546,328676,2019-12-31,Singapore,Premium Sticker Mart,Kaggle,NaN
98547,328677,2019-12-31,Singapore,Premium Sticker Mart,Kaggle Tiers,NaN
98548,328678,2019-12-31,Singapore,Premium Sticker Mart,Kerneler,NaN


In [235]:
def create_sinusoidal_transformation_year_month_day(df, col_name, year, month, day, period):
    """
    Adds sinusoidal transformation columns (sin and cos) for year, month, day.
    """
    df[f'{col_name}_sin'] = np.sin(2 * np.pi * df[year] * df[month] * df[day] / period)
    df[f'{col_name}_cos'] = np.cos(2 * np.pi * df[year] * df[month] * df[day] / period)
    return df


In [236]:
def create_time_features(data: pd.DataFrame, date_col='date'):
    df = data.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col).reset_index(drop=True)

    # Time-based features
    df['year'] = df[date_col].dt.year
    df['month'] = df[date_col].dt.month
    df['day'] = df[date_col].dt.day

    df = create_sinusoidal_transformation_year_month_day(df, 'date', "year", "month", "day", 12)

    return df

### Imputation

Monthly average sales per product per store per country

In [237]:
# create temporal features for imputation
df_train = create_time_features(df_train)

In [238]:
def imputation(data, group_by):
    df = data.copy()

    df['num_sold'] = df['num_sold'].fillna(0)

    df_temp = df.groupby(group_by)['num_sold'].mean().reset_index(name='avg_sold').round(0)

    df_merge = pd.merge(df_train, df_temp, how='left', on=group_by)

    df_merge['num_sold'] = np.where(df_merge['num_sold'].isna(), df_merge['avg_sold'], df_merge['num_sold'])

    return df_merge

In [206]:
group_by = ['country','store','product', 'month']
df_train = imputation(df_train, group_by)

In [207]:
df_train.head()

,id,date,country,store,product,num_sold,year,month,day,date_sin,date_cos,avg_sold
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,0.0,2010,1,1,8.721095e-14,-1.0,0.0
1,65,2010-01-01,Norway,Stickers for Less,Holographic Goose,579.0,2010,1,1,8.721095e-14,-1.0,429.0
2,64,2010-01-01,Norway,Discount Stickers,Kerneler Dark Mode,911.0,2010,1,1,8.721095e-14,-1.0,748.0
3,63,2010-01-01,Norway,Discount Stickers,Kerneler,791.0,2010,1,1,8.721095e-14,-1.0,581.0
4,62,2010-01-01,Norway,Discount Stickers,Kaggle Tiers,1525.0,2010,1,1,8.721095e-14,-1.0,1129.0


In [208]:
df_train.isna().sum()

id          0
date        0
country     0
store       0
product     0
num_sold    0
year        0
month       0
day         0
date_sin    0
date_cos    0
avg_sold    0
dtype: int64

## Feature engineering

In [209]:
# lag features
def create_lag_features(data: pd.DataFrame, target_col='num_sold'):
    df = data.copy()
    df[target_col + '_lag_1'] =  df[target_col].shift(1)
    df[target_col + '_lag_2'] =  df[target_col].shift(2)
    df[target_col + '_lag_3'] =  df[target_col].shift(3)

    df[target_col + '_rolling_mean_3'] = df[target_col].rolling(window=3).mean()
    df[target_col + '_rolling_mean_5'] = df[target_col].rolling(window=5).mean()
    df[target_col + '_rolling_mean_7'] = df[target_col].rolling(window=7).mean()

    lag_cols = [col for col in df.columns if re.search('lag', col) or re.search('rolling', col)]
    for col in lag_cols:
        df[col] = df[col].fillna(df[target_col])

    return df

def generate_lag_features_by_group(df, group_cols, target_col='num_sold'):
    # Group by the specified columns and apply lag feature creation
    grouped = df.groupby(group_cols, group_keys=False)
    df_with_features = grouped.apply(create_lag_features, target_col=target_col).reset_index(drop=True)
    
    return df_with_features

In [210]:
group_cols = ['country', 'store', 'product']
df_train = generate_lag_features_by_group(df_train, group_cols)

C:\Users\sean.chang\AppData\Local\Temp\ipykernel_27276\2212948589.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_with_features = grouped.apply(create_lag_features, target_col=target_col).reset_index(drop=True)


In [211]:
df_train

,id,date,country,store,product,num_sold,year,month,day,date_sin,date_cos,avg_sold,num_sold_lag_1,num_sold_lag_2,num_sold_lag_3,num_sold_rolling_mean_3,num_sold_rolling_mean_5,num_sold_rolling_mean_7
0,0,2010-01-01,Canada,Discount Stickers,Holographic Goose,0.0,2010,1,1,8.721095e-14,-1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
1,65,2010-01-01,Norway,Stickers for Less,Holographic Goose,579.0,2010,1,1,8.721095e-14,-1.0,429.0,579.0,579.0,579.0,579.000000,579.0,579.000000
2,64,2010-01-01,Norway,Discount Stickers,Kerneler Dark Mode,911.0,2010,1,1,8.721095e-14,-1.0,748.0,911.0,911.0,911.0,911.000000,911.0,911.000000
3,63,2010-01-01,Norway,Discount Stickers,Kerneler,791.0,2010,1,1,8.721095e-14,-1.0,581.0,791.0,791.0,791.0,791.000000,791.0,791.000000
4,62,2010-01-01,Norway,Discount Stickers,Kaggle Tiers,1525.0,2010,1,1,8.721095e-14,-1.0,1129.0,1525.0,1525.0,1525.0,1525.000000,1525.0,1525.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230125,230065,2016-12-31,Finland,Premium Sticker Mart,Holographic Goose,349.0,2016,12,31,1.311460e-11,1.0,282.0,338.0,310.0,344.0,332.333333,324.0,314.428571
230126,230064,2016-12-31,Finland,Stickers for Less,Kerneler Dark Mode,1012.0,2016,12,31,1.311460e-11,1.0,757.0,1026.0,978.0,876.0,1005.333333,943.0,899.000000
230127,230063,2016-12-31,Finland,Stickers for Less,Kerneler,846.0,2016,12,31,1.311460e-11,1.0,709.0,844.0,757.0,790.0,815.666667,789.6,763.857143
230128,230060,2016-12-31,Finland,Stickers for Less,Holographic Goose,289.0,2016,12,31,1.311460e-11,1.0,237.0,303.0,293.0,315.0,295.000000,286.0,272.142857


### Encoding

one-hot encoding

In [212]:
categorical_col = df_train.select_dtypes('object').columns.to_list()
categorical_col

['country', 'store', 'product']

In [213]:
df_encoded = pd.get_dummies(df_train, columns=categorical_col, drop_first=False, dtype=int)

In [214]:
df_encoded.head()

,id,date,num_sold,year,month,day,date_sin,date_cos,avg_sold,num_sold_lag_1,...,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
0,0,2010-01-01,0.0,2010,1,1,8.721095e-14,-1.0,0.0,0.0,...,0,0,1,0,0,1,0,0,0,0
1,65,2010-01-01,579.0,2010,1,1,8.721095e-14,-1.0,429.0,579.0,...,1,0,0,0,1,1,0,0,0,0
2,64,2010-01-01,911.0,2010,1,1,8.721095e-14,-1.0,748.0,911.0,...,1,0,1,0,0,0,0,0,0,1
3,63,2010-01-01,791.0,2010,1,1,8.721095e-14,-1.0,581.0,791.0,...,1,0,1,0,0,0,0,0,1,0
4,62,2010-01-01,1525.0,2010,1,1,8.721095e-14,-1.0,1129.0,1525.0,...,1,0,1,0,0,0,0,1,0,0


### Standardization

In [215]:
def standardization(data: pd.DataFrame):
    df = data.copy()
    scaler = StandardScaler()
    df_standardize = scaler.fit_transform(df)
    df_standardize = pd.DataFrame(df_standardize, columns=df.columns, index=df.index)
    return df_standardize

In [216]:
df_encoded = df_encoded.set_index('id')
df_encoded = df_encoded.drop('date', axis=1)
df_standardize = standardization(df_encoded)

## Model

train_test_split

In [225]:
target_col = 'num_sold'
X_train, X_valid, y_train, y_valid = train_test_split(df_standardize.drop(target_col, axis=1), df_standardize[target_col], test_size=0.2)

In [218]:
X_train

,year,month,day,date_sin,date_cos,avg_sold,num_sold_lag_1,num_sold_lag_2,num_sold_lag_3,num_sold_rolling_mean_3,...,country_Norway,country_Singapore,store_Discount Stickers,store_Premium Sticker Mart,store_Stickers for Less,product_Holographic Goose,product_Kaggle,product_Kaggle Tiers,product_Kerneler,product_Kerneler Dark Mode
id,,,,,,,,,,,,,,,,,,,,,
155814,0.499560,0.718364,1.394111,-0.000313,0.835350,-0.199166,-0.082832,-0.186964,-0.236177,-0.053735,...,-0.447214,-0.447214,-0.707107,-0.707107,1.414214,-0.5,-0.5,-0.5,-0.5,2.0
8065,-1.500245,-1.021359,1.734995,-0.000313,-1.672416,-1.063873,-1.040300,-1.040266,-1.040207,-1.046630,...,-0.447214,-0.447214,-0.707107,1.414214,-0.707107,2.0,-0.5,-0.5,-0.5,-0.5
152098,0.499560,0.428411,0.144201,1.722999,-1.045474,0.387176,0.501484,0.338033,0.404443,0.494855,...,-0.447214,2.236068,-0.707107,1.414214,-0.707107,-0.5,-0.5,-0.5,2.0,-0.5
28449,-1.500245,1.298272,-0.310311,-0.000313,-1.672416,-0.105885,-0.130561,-0.146469,-0.160980,-0.110001,...,-0.447214,-0.447214,-0.707107,-0.707107,1.414214,-0.5,-0.5,-0.5,-0.5,2.0
188343,0.999511,0.718364,0.939598,-0.000313,0.835350,-0.046658,-0.312798,-0.205766,-0.324389,-0.264732,...,2.236068,-0.447214,1.414214,-0.707107,-0.707107,-0.5,-0.5,-0.5,2.0,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140567,0.499560,-0.731405,-0.537567,-1.723624,-1.045474,-0.077752,-0.019193,-0.103081,-0.016371,-0.054705,...,-0.447214,2.236068,1.414214,-0.707107,-0.707107,-0.5,-0.5,2.0,-0.5,-0.5
128554,-0.000391,1.298272,1.507739,1.989596,-0.418533,-0.732204,-0.690289,-0.671466,-0.688805,-0.697395,...,-0.447214,-0.447214,1.414214,-0.707107,-0.707107,-0.5,-0.5,-0.5,-0.5,2.0
163392,0.499560,1.588226,0.598714,-0.000313,0.835350,0.483419,0.277304,0.397330,0.268510,0.373593,...,-0.447214,-0.447214,-0.707107,1.414214,-0.707107,-0.5,-0.5,2.0,-0.5,-0.5


xgboost

In [219]:
model = XGBRFRegressor(
    n_estimators=100,        # Number of trees
    max_depth=5,             # Maximum depth of each tree
    learning_rate=0.1,       # Learning rate (shrinkage factor)
    subsample=0.8,           # Subsample ratio for training instances
    colsample_bynode=0.8,    # Subsample ratio for columns at each tree node
    random_state=42          # For reproducibility
)

In [221]:
model.fit(X_train, y_train)

XGBRFRegressor(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bytree=None, device=None,
               early_stopping_rounds=None, enable_categorical=False,
               eval_metric=None, feature_types=None, gamma=None,
               grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.1, max_bin=None,
               max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=5, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=100, n_jobs=None,
               num_parallel_tree=None, objective='reg:squarederror',
               random_state=42, ...)

In [222]:
y_pred = model.predict(X_valid)

In [227]:
mse = mean_squared_error(y_valid, y_pred)
mse

1.0111703629069801